In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-20 13:59:52.401331: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 13:59:52.632914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
from transformers import AdamW
from adapters import AdapterConfig,LoRAConfig
from adapters.composition import Stack

adapter_name= "task-adapter-after-lora-mlm"

adapter_config = LoRAConfig(r=8, alpha=16)

model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/lora-source-mlm")
model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/lora-mlm-target")
model.add_adapter(adapter_name, config=adapter_config)

model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.train_adapter([adapter_name])


In [6]:
model.active_adapters = Stack("lora-source-mlm", adapter_name)

fn.print_trainable_parameters(model)

trainable params: 2608305 || all params: 69266097 || trainable%: 3.7656301032812634


In [7]:
model.adapter_summary(as_dict=True)

[{'name': 'lora-source-mlm',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': False,
  '%param': 0.22219650503413957},
 {'name': 'lora-mlm-target',
  'architecture': 'lora',
  'active': False,
  '#param': 147456,
  'train': False,
  '%param': 0.22219650503413957},
 {'name': 'task-adapter-after-lora-mlm',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': True,
  '%param': 0.22219650503413957},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [8]:
source_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 75013
})

In [9]:
from transformers import TrainingArguments, EvalPrediction,default_data_collator
from adapters import AdapterTrainer
eval_data = None
batch_size = 32

logging_steps = len(source_data) // batch_size

training_args = TrainingArguments(
        
        output_dir=f"{config.Config.RESULTS_SAVE_PATH}/task_after_lora_mlm/results",                 # Where to store the output (checkpoints and predictions)
        num_train_epochs=8,                     # Total number of training epochs
        per_device_train_batch_size=batch_size,         # Batch size for training
        per_device_eval_batch_size=batch_size,          # Batch size for evaluation
        warmup_steps=500,                       # Number of warmup steps for learning rate scheduler
        learning_rate=1e-4,
        weight_decay=0.01,                      # Strength of weight decay
        logging_dir=f"{config.Config.RESULTS_SAVE_PATH}/task_after_lora_mlm/logs",                   # Directory for storing logs
        logging_steps=logging_steps,                       # Log every X updates steps
        remove_unused_columns=False,
        evaluation_strategy="steps" if eval_data is not None else "no",            # Evaluate model every X steps
        eval_steps=logging_steps,                         # Number of steps to perform evaluation
        save_steps=logging_steps,                         # Save checkpoint every X steps
        save_total_limit=2,                     # Limit the total amount of checkpoints
        load_best_model_at_end=True if eval_data is not None else False,            # Load the best model when finished training
        report_to="none"                        # Do not report to any online service
    ) 
trainer = AdapterTrainer(
        model=model,                           # The instantiated 🤗 Transformers model to be trained
        args=training_args,                    # Training arguments, defined above
        train_dataset=source_data,           # Training dataset
        )

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [10]:
trainer.train()


  0%|          | 0/18760 [00:00<?, ?it/s]

{'loss': 0.891, 'learning_rate': 8.990142387732749e-05, 'epoch': 1.0}
{'loss': 0.7697, 'learning_rate': 7.706462212486309e-05, 'epoch': 2.0}
{'loss': 0.7395, 'learning_rate': 6.422782037239868e-05, 'epoch': 3.0}
{'loss': 0.7227, 'learning_rate': 5.1391018619934285e-05, 'epoch': 4.0}
{'loss': 0.7114, 'learning_rate': 3.855421686746988e-05, 'epoch': 5.0}
{'loss': 0.7001, 'learning_rate': 2.5717415115005478e-05, 'epoch': 6.0}
{'loss': 0.6938, 'learning_rate': 1.2880613362541074e-05, 'epoch': 7.0}
{'loss': 0.6861, 'learning_rate': 4.381161007667032e-08, 'epoch': 8.0}
{'train_runtime': 1731.1819, 'train_samples_per_second': 346.644, 'train_steps_per_second': 10.837, 'train_loss': 0.7393012342422501, 'epoch': 8.0}


TrainOutput(global_step=18760, training_loss=0.7393012342422501, metrics={'train_runtime': 1731.1819, 'train_samples_per_second': 346.644, 'train_steps_per_second': 10.837, 'train_loss': 0.7393012342422501, 'epoch': 8.0})

In [11]:
model.active_adapters = Stack("lora-mlm-target", adapter_name)

model.adapter_summary(as_dict=True)

[{'name': 'lora-source-mlm',
  'architecture': 'lora',
  'active': False,
  '#param': 147456,
  'train': False,
  '%param': 0.22219650503413957},
 {'name': 'lora-mlm-target',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': False,
  '%param': 0.22219650503413957},
 {'name': 'task-adapter-after-lora-mlm',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': True,
  '%param': 0.22219650503413957},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [12]:
import numpy as np
from transformers import EvalPrediction
from sklearn.metrics import accuracy_score, f1_score,precision_recall_fscore_support


def compute_metrics(pred:EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=test_target_data,
    compute_metrics=compute_metrics,
)
eval_trainer.evaluate()

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/247 [00:00<?, ?it/s]

{'eval_loss': 1.0387978553771973,
 'eval_accuracy': 0.4681174089068826,
 'eval_f1': 0.42692643704349104,
 'eval_precision': 0.5401774675329262,
 'eval_recall': 0.4681174089068826,
 'eval_runtime': 3.4299,
 'eval_samples_per_second': 576.103,
 'eval_steps_per_second': 72.013}

In [28]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name,with_head=True)